# Node2Vec

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

## Baixando e instalando bibliotecas

In [ ]:
!git clone https://github.com/shenweichen/GraphEmbedding

Cloning into 'GraphEmbedding'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 164 (delta 4), reused 10 (delta 1), pack-reused 146
Receiving objects: 100% (164/164), 669.54 KiB | 20.29 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [ ]:
!pip install tensorflow==1.4.0

     |████████████████████████████████| 41.2MB 120kB/s 
     |████████████████████████████████| 1.7MB 39.7MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107222 sha256=38ae4041e3df3999176f3df1e5bf741d6ecb64dc76af259b08a327283afe95bb
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.2.1
    Uninstalling bleach-3.2.1:
      Successfully uninstalled bleach-3.2.1
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0


In [ ]:
!mv GraphEmbedding/* .

# Carregando as Redes

In [ ]:
!gdown --id 1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0

Downloading...
From: https://drive.google.com/uc?id=1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0
To: /content/datasets.tar.gz
22.7MB [00:00, 86.2MB/s]


In [ ]:
!tar -xzvf datasets.tar.gz

datasets_runs/
datasets_runs/run_1_google_news_5w1h_graph_hin.nx
datasets_runs/run_6_40er_5w1h_graph_hin.nx
datasets_runs/run_4_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_5_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_google_news_5w1h_graph_hin.nx
datasets_runs/run_5_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_2_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_7_40er_5w1h_graph_hin.nx
datasets_runs/run_9_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_8_google_news_5w1h_graph_hin.nx
datasets_runs/run_10_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_2_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_8_40er_5w1h_graph_hin.nx
datasets_runs/run_6_bbc_5w1h_graph_hin.nx
datasets_runs/run_4_google_news_5w1h_graph_hin.nx
datasets_runs/run_2_google_news_5w1h_graph_hin.nx
datasets_runs/run_7_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_4_gold_standard_

In [ ]:
from os import listdir
from os.path import isfile, join
path_datasets = 'datasets_runs/'
network_files = [f for f in listdir(path_datasets) if isfile(join(path_datasets, f))]
print(network_files)

['run_1_google_news_5w1h_graph_hin.nx', 'run_6_40er_5w1h_graph_hin.nx', 'run_4_bbc_5w1h_graph_hin.nx', 'run_8_gold_standard_5w1h_graph_hin.nx', 'run_5_bbc_5w1h_graph_hin.nx', 'run_9_google_news_5w1h_graph_hin.nx', 'run_5_gold_standard_5w1h_graph_hin.nx', 'run_2_bbc_5w1h_graph_hin.nx', 'run_9_news_cluster_5w1h_graph_hin.nx', 'run_7_40er_5w1h_graph_hin.nx', 'run_9_gold_standard_5w1h_graph_hin.nx', 'run_8_google_news_5w1h_graph_hin.nx', 'run_10_bbc_5w1h_graph_hin.nx', 'run_8_news_cluster_5w1h_graph_hin.nx', 'run_2_news_cluster_5w1h_graph_hin.nx', 'run_8_40er_5w1h_graph_hin.nx', 'run_6_bbc_5w1h_graph_hin.nx', 'run_4_google_news_5w1h_graph_hin.nx', 'run_2_google_news_5w1h_graph_hin.nx', 'run_7_gold_standard_5w1h_graph_hin.nx', 'run_4_gold_standard_5w1h_graph_hin.nx', 'run_5_40er_5w1h_graph_hin.nx', 'run_3_gold_standard_5w1h_graph_hin.nx', 'run_4_40er_5w1h_graph_hin.nx', 'run_5_google_news_5w1h_graph_hin.nx', 'run_10_news_cluster_5w1h_graph_hin.nx', 'run_10_40er_5w1h_graph_hin.nx', 'run_9_40

# Treinando e avaliando o Node2Vec

In [ ]:
from ge import Node2Vec
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC


experimental_results = []
emb_sizes = [ 64 ]
parameters = [(0.5,1),(1,2),(0.25,0.25)]

for network_file in tqdm(network_files):

  for emb_size in emb_sizes:

    for par in parameters:

      p=par[0]
      q=par[1]

      G = nx.read_gpickle(path_datasets+network_file)
      model = Node2Vec(G, walk_length = 10, num_walks = 80, p = p, q = q, workers = 1)
      model.train(window_size=5,iter=3,embed_size=emb_size)# train model
      embeddings = model.get_embeddings()# get embedding vectors    


      # gerando features e classificando
      X_train = []
      Y_train = []
      X_test = []
      Y_test = []

      for node in G.nodes():
        if ':event' in node:
          
          if 'train' in G.nodes[node]:
            X_train.append(embeddings[node])
            Y_train.append(G.nodes[node]['label'])
          if 'test' in G.nodes[node]:
            X_test.append(embeddings[node])
            Y_test.append(G.nodes[node]['label'])


      X_train = np.array(X_train)
      X_test = np.array(X_test)

      clf = LinearSVC(tol=1e-5).fit(X_train, Y_train)
      y_pred = clf.predict(X_test)


      f1_macro = f1_score(Y_test, y_pred, average='macro')
      acc = accuracy_score(Y_test, y_pred)

      print('--->' ,network_file,emb_size,p,q,'f1_macro',f1_macro,'acc',acc)
      experimental_results.append((network_file,emb_size,p,q,'f1_macro',f1_macro,'acc',acc,Y_test,y_pred))


Preprocess transition probs...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


Learning embedding vectors...
Learning embedding vectors done!
---> run_1_google_news_5w1h_graph_hin.nx 64 0.5 1 f1_macro 0.4995076352219209 acc 0.56
Preprocess transition probs...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


Learning embedding vectors...
Learning embedding vectors done!
---> run_1_google_news_5w1h_graph_hin.nx 64 1 2 f1_macro 0.6632914704343277 acc 0.68
Preprocess transition probs...


...

Learning embedding vectors...
Learning embedding vectors done!
---> run_3_40er_5w1h_graph_hin.nx 64 0.5 1 f1_macro 0.5935353535353536 acc 0.6875
Preprocess transition probs...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished


Learning embedding vectors...
Learning embedding vectors done!
---> run_3_40er_5w1h_graph_hin.nx 64 1 2 f1_macro 0.6202020202020201 acc 0.71875
Preprocess transition probs...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished


Learning embedding vectors...
Learning embedding vectors done!
---> run_3_40er_5w1h_graph_hin.nx 64 0.25 0.25 f1_macro 0.6606060606060606 acc 0.78125



In [ ]:
import pandas as pd
df_results = pd.DataFrame(experimental_results)
df_results

,0,1,2,3,4,5,6,7,8,9
0,run_1_google_news_5w1h_graph_hin.nx,64,0.50,1.00,f1_macro,0.499508,acc,0.560000,"[Equifax breach, IPhone-X Green-Line, Star War...","[Vegas Shooting, IPhone-X Green-Line, Zimbabwe..."
1,run_1_google_news_5w1h_graph_hin.nx,64,1.00,2.00,f1_macro,0.663291,acc,0.680000,"[Equifax breach, IPhone-X Green-Line, Star War...","[Vegas Shooting, IPhone-X Green-Line, Star War..."
2,run_1_google_news_5w1h_graph_hin.nx,64,0.25,0.25,f1_macro,0.626799,acc,0.720000,"[Equifax breach, IPhone-X Green-Line, Star War...","[Zimbabwe, IPhone-X Green-Line, Star Wars: Bat..."
3,run_6_40er_5w1h_graph_hin.nx,64,0.50,1.00,f1_macro,0.778860,acc,0.843750,"[business, world, world, business, business, t...","[business, world, world, business, business, w..."
4,run_6_40er_5w1h_graph_hin.nx,64,1.00,2.00,f1_macro,0.804233,acc,0.875000,"[business, world, world, business, business, t...","[world, world, world, business, business, worl..."
...,...,...,...,...,...,...,...,...,...,...
145,run_7_news_cluster_5w1h_graph_hin.nx,64,1.00,2.00,f1_macro,0.470200,acc,0.591503,"[harambe, The_New_Infinity_War_Trailer, pittsb...","[harambe, thousands_flee_fireworks_explosion_a..."
146,run_7_news_cluster_5w1h_graph_hin.nx,64,0.25,0.25,f1_macro,0.479167,acc,0.591503,"[harambe, The_New_Infinity_War_Trailer, pittsb...","[harambe, thousands_flee_fireworks_explosion_a..."
147,run_3_40er_5w1h_graph_hin.nx,64,0.50,1.00,f1_macro,0.593535,acc,0.687500,"[business, world, world, business, technology,...","[world, world, world, business, business, busi..."
148,run_3_40er_5w1h_graph_hin.nx,64,1.00,2.00,f1_macro,0.620202,acc,0.718750,"[business, world, world, business, technology,...","[business, world, world, business, world, busi..."


In [ ]:
df_results.to_excel('node2vec_results_r1.xls')